<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/GS_v0_2_con_vocab_allbase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/par_esp_interventions_l1_l13.csv')

<ipython-input-2-f7788a307fca>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('/content/drive/MyDrive/par_esp_interventions_l1_l13.csv')


In [3]:
import pickle

with open('/content/drive/MyDrive/vocabulary.voc', 'rb') as f:
    inter = pickle.load(f)

In [4]:
base=base.rename(columns={"party": "partido"})

In [5]:
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])
     

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# Elimino de la base las intervenciones del Presidente de la cámara
base=base[(base['role'] != 'PRESIDENTE, PRESIDENTA') | (base['role'] != 'PRESIDENTE DEL CONGRESO DE LOS DIPUTADOS')]  

In [8]:
# Eliminos nans de la base
base=base[base['intervention'].notna()]

In [9]:
# Esta celda tarda unos 30 minutos en correr
porter_stemmer = PorterStemmer()

lista = []
base['tokens']=""
for row in base.values:
    #elimino puntuación
    non_punctuation = row[2].replace('-',' ')
    non_punctuation = non_punctuation.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(non_punctuation)

    #elimino stopwords
    tokens=[w for w in word_tokens if not w.lower() in stopwords]

    #stemming
    stemmers = [porter_stemmer.stem(word) for word in tokens]
    row[18] = [stem for stem in stemmers if stem.isalpha() and len(stem) > 1]
    lista.append(row)

<ipython-input-9-fb6ca65ec766>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['tokens']=""


In [10]:
base = pd.DataFrame(lista, columns=base.columns)

In [11]:
base['tokens'] = [str(token) for token in base['tokens']]

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(2,2) ,vocabulary=inter)
matrix = vect.fit_transform(base['tokens'])

In [13]:
len(vect.vocabulary_)

270230

In [14]:
base.columns

Index(['orden', 'namex', 'intervention', 'fecha', 'legislatura', 'pleno',
       'role', 'oradorx', 'namey', 'partido', 'province', 'congreso', 'senado',
       'total', 'oradory', 'cataluna', 'estatut', 'inter_raw', 'tokens'],
      dtype='object')

In [15]:
# Esta celda tarda unos 70 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=30, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)  
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, n_components=30,
                          random_state=0)

In [16]:
import numpy as np
from numpy import argsort
import statistics
from statistics import mode
from scipy.special import logsumexp
import scipy as sp

# Bases nuevas

In [17]:
disc_weights=matrix*lda.components_.transpose()
disc_weights.shape

(447525, 30)

In [18]:
column_names = [str(i) for i in range(1, 31)]

df_weights = pd.DataFrame(disc_weights, columns=column_names)

select=[ 'intervention'	, 'fecha' , 'partido', 'namex']
df_weights = pd.concat([base[select], df_weights], axis=1)

In [19]:
df_weights.to_csv('df_weights.csv')

In [20]:
import scipy as sp
from scipy.special import logsumexp

def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out
  
def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

In [21]:
frex_df=pd.DataFrame(frex(lda))
disc_frex=matrix*frex_df.transpose()

In [23]:
df_frex = pd.DataFrame(disc_weights, columns=column_names)

select=[ 'intervention'	, 'fecha' , 'partido', 'namex']
df_frex = pd.concat([base[select], df_frex], axis=1)
     

df_frex.to_csv('df_frex.csv')

# Topics por weights

In [ ]:
disc_weights=matrix*lda.components_.transpose()

In [ ]:
base['Topic_weight']=disc_weights.argmax(axis=1)+1

In [ ]:
base.fecha=pd.to_datetime(base.fecha, infer_datetime_format=True)

In [ ]:
# Por LEGISLATURA por WEIGHTS
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas: 
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_weight'].value_counts().sort_index()/base_legislatura['Topic_weight'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_weights.csv')

In [ ]:
# Por AÑO por weights
topics_por_año=pd.DataFrame()
for x in range (1979, 2020): 
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_weight'].value_counts().sort_index()/base_year['Topic_weight'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_weights.csv')

# FREX

In [ ]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [ ]:
frex_df=pd.DataFrame(frex(lda))

In [ ]:
# Se crea una columna con el topic que se asigna por frex a cada discurso de la base
base['id']=range(0,len(base))
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)+1

In [ ]:
# Por LEGISLATURA por FREX
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas: 
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_FREX'].value_counts().sort_index()/base_legislatura['Topic_FREX'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_frex.csv')

In [ ]:
# Por AÑO por FREX
topics_por_año=pd.DataFrame()
for x in range (1979, 2020): 
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_FREX'].value_counts().sort_index()/base_year['Topic_FREX'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_frex.csv')

<ipython-input-20-70637a8bf3e9>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
<ipython-input-20-70637a8bf3e9>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
<ipython-input-20-70637a8bf3e9>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
<ipython-input-20-70637a8bf3e9>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsisten

# Bigramas por weights

In [ ]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [ ]:
bigramas.to_csv('bigramas_por_weights_30topics.csv')

# Bigramas por frex

In [ ]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [ ]:
bigramas.to_csv('bigramas_por_frex_30topics.csv')

# Bigramas para 10 Topics

In [ ]:
# Esta celda tarda unos 25 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)  
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, random_state=0)

In [ ]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [ ]:
bigramas.to_csv('bigramas_por_weights_10topics.csv')

In [ ]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [ ]:
bigramas.to_csv('bigramas_por_frex_10topics.csv')